In [1]:
# context is added to the prompt by retrieving data from an external source, in this case, Amazon Kendra, 
# a search engine service provided by AWS.
# for this exercise, customer service interaction notes have been generated using an LLM.
# these notes are for a fictitious ticketing company named 'ticketmom'. 
# the interactions have been indexed by Kendra and are ready for search.

# the code below retrieves search results from Kendra, injects them into a prompt, then asks a question 
# about the content.

In [2]:
import boto3

from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import PromptTemplate
from langchain_community.retrievers import AmazonKendraRetriever
from langchain.prompts.few_shot import FewShotPromptTemplate

In [3]:
# reference to the foundation model via amazon bedrock.
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # claude is awesome!    
    model_kwargs={"temperature": 0.95},                  # parameters vary by model
#     guardrails={                                         # guardrails filter prompts and responses
#         "guardrailIdentifier": "z0x1jv6cnbct", 
#         "guardrailVersion": "1"
#     }
)

In [4]:
# the following code is used to look up the id of the kendra index
# you probably don't need to do this in your own AWS account.

# create connection to AWS API using boto3
client = boto3.client('kendra')

# return a list of all indices
response = client.list_indices()

# iterate through the indices until you find the one named 'ticketmom'
# ..then assign the Id to index_id
index_id = None
for index in response['IndexConfigurationSummaryItems']:
    if index['Name'] == 'ticketmom':
        index_id = index['Id']

print(f"Amazon Kendra Index ID: {index_id}")

Amazon Kendra Index ID: 02e8ed7b-46c2-4bde-9b97-3f9491a7f97c


In [5]:
# retrievers are langchain objects which retrieve from external data stores.
retriever = AmazonKendraRetriever(
    index_id=index_id,
    top_k = 5 # the number of search result documents to return
)

In [9]:
# we are using a few-shot prompt template because it easily injects a list of records
# into a prompt. we are not doing few-shot prompting for this use case.

# this is the search term we will use when querying kendra
customer_service_issue = 'computer error'

# generates the prompt and injects the search results
prompt = FewShotPromptTemplate(
    examples=retriever.invoke(customer_service_issue),
    example_prompt=PromptTemplate(
        input_variables=["page_content"],
        template="{page_content}\n",
    ),
    suffix="""
act as an expert in customer service. 

assess the preceding customer interactions to identify how ticketmom could 
deliver better quality service.. with the goal of creating happier customers.

articulate your suggestion very succinctly, with three bullet points one sentence each.

"""
)

# let's have a look at the prompt:
print(prompt.format())

Document Title: Neon Dreams Festival.txt
Document Excerpt: 
She was adamant that she did not want or need 6 tickets. I apologized for the confusion and double charge. After reviewing the order details, I saw that the first order of 2 tickets was placed from her computer, and the second order of 4 tickets was placed from her mobile device an hour later. Jennifer insisted she only meant to purchase the 4 mobile tickets, but wasn't sure why the first computer order went through as well. She got flustered trying to explain what happened and blamed our "terrible website" for the mixup. I tried to de-escalate the situation by taking full responsibility and initiating a refund for the unintended 2 ticket order from her computer. However, she was still extremely unhappy about being overcharged initially. After some back and forth, I issued her a one-time 15% discount on the 4 mobile tickets she wanted to keep as a courtesy. I also offered to have the 4 tickets reprinted as a new combined order

In [10]:
chain = prompt | model | StrOutputParser()

In [11]:
response = chain.invoke(input={})

print(response)

To deliver better quality service and create happier customers, TicketMom should:

• Ensure accurate customer information is captured and maintained to avoid issues like sending tickets to the wrong email address. 

• Prioritize improving website/app functionality and performance to prevent technical glitches that frustrate customers.

• Train customer service agents on effective de-escalation techniques to better manage irate customers and resolve issues calmly.
